In [ ]:
import os
import pandas as pd
from PIL import Image
import shutil
from tqdm import tqdm

class Config():
    csv_path = r"E:\ISICDM2025\competition_image\isicdm2025.csv"
    image_root = r"E:\ISICDM2025\ISICDM2025_dataset\images"  # 原始图像位置（train/val/test）
    output_dir = r"E:\ISICDM2025\yolo_dataset"  # 输出 YOLO 格式数据集

# 创建输出目录
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(Config.output_dir, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(Config.output_dir, 'labels', split), exist_ok=True)

df = pd.read_csv(Config.csv_path)

categories = sorted(df['category_id'].unique())
nc = len(categories)
names = [f"class{i}" for i in categories]  # 可根据实际类别名替换

# 按 split 分组处理
for _, row in tqdm(df.iterrows()):
    split = row['split']
    filename = row['anonymous_filename']
    img_path = os.path.join(Config.image_root, split, filename)
    
    # 目标路径
    dst_img_path = os.path.join(Config.output_dir, 'images', split, filename)
    label_path = os.path.join(Config.output_dir, 'labels', split, filename.replace('.png', '.txt'))

    # 计算归一化坐标
    width = row['width']
    height = row['height']
    x_center = (row['xmin'] + row['xmax']) / 2 / width
    y_center = (row['ymin'] + row['ymax']) / 2 / height
    w_norm = (row['xmax'] - row['xmin']) / width
    h_norm = (row['ymax'] - row['ymin']) / height
    cls_id = row['category_id']

    # 写入 YOLO 格式标签（追加模式，一张图可能有多个目标）
    with open(label_path, 'a') as f:
        f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

In [ ]:
# 生成 data.yaml
yaml_content = f"""path: {Config.output_dir}
train: images/train
val: images/val
test: images/test

nc: {nc}
names: {names}
"""

with open(os.path.join(Config.output_dir, 'data.yaml'), 'w') as f:
    f.write(yaml_content)

In [ ]:
from ultralytics import YOLO

pretrained_weights = r"E:\ISICDM2025\pth\yolov8.pt"  # 你训练好的权重
data_yaml = r"E:\ISICDM2025\yolo_dataset\data.yaml"

model = YOLO(pretrained_weights)  # 自动识别架构

# 微调训练
model.train(
    data=data_yaml,
    epochs=80,               
    imgsz=384,              
    batch=32,                
    name='384yolov8',       
    device=0,               
    cache=False,             
    hsv_h=0.0,               
    hsv_s=0.0,
    hsv_v=0.0,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.0
)